# Content translation from English to Thai 

follow steps according to https://paws-public.wmflabs.org/paws-public/User:Isaac_(WMF)/Content%20Translation%20Example.ipynb#Alternative-view-of-the-data-via-Pandas

## Initialization

In [1]:
#imports
import gzip
import json
import re

import mwapi
import pandas as pd

import mwviews
from mwviews.api import PageviewsClient

In [2]:
session = mwapi.Session(host='https://en.wikipedia.org',
                        user_agent='mwapi (python) -- outreachy content translation')

In [3]:
# articles translated from English -> Thai
parameters = {'action':'query',
              'format':'json',
              'list':'cxpublishedtranslations',
              'from':'en',
              'to':'th',
              'limit':500,
              'offset':2500}

## Explore data

In [4]:
res = session.get(parameters)

In [5]:
df = pd.DataFrame(res['result']['translations'])
df.head(10)

,publishedDate,sourceLanguage,sourceRevisionId,sourceTitle,sourceURL,stats,targetLanguage,targetRevisionId,targetTitle,targetURL,translationId
0,20180616090318,en,846093806,Sonia Scurfield,//en.wikipedia.org/wiki/Sonia Scurfield,"{'any': 0.41319606925597, 'human': 0.313991576...",th,7678509,ซอนย่า สเกอร์ฟิลด์,//th.wikipedia.org/wiki/ซอนย่า สเกอร์ฟิลด์,491563
1,20180616092527,en,844984088,Andrew Massey (conductor),//en.wikipedia.org/wiki/Andrew Massey (conductor),"{'any': 0.92383292383292, 'human': 0.883292383...",th,7678529,แอนดี แมสซีย์ (วาทยากร),//th.wikipedia.org/wiki/แอนดี แมสซีย์ (วาทยากร),491571
2,20180616093431,en,845176394,Sam Moore (publisher),//en.wikipedia.org/wiki/Sam Moore (publisher),"{'any': 0.42849223946785, 'human': 0.428492239...",th,7678540,ซัม มัวร์ (ผู้เผยแพร่),//th.wikipedia.org/wiki/ซัม มัวร์ (ผู้เผยแพร่),491580
3,20180901233542,en,845016481,André Desvages,//en.wikipedia.org/wiki/André Desvages,"{'any': 0.59139784946237, 'human': 0.479569892...",th,7678579,อ็องเดร เดอวาก,//th.wikipedia.org/wiki/อ็องเดร เดอวาก,491589
4,20180616095938,en,845590955,Johnnie Keyes,//en.wikipedia.org/wiki/Johnnie Keyes,"{'any': 0.86713995943205, 'human': 0.735294117...",th,7678608,จอห์นนี่ คียส์,//th.wikipedia.org/wiki/จอห์นนี่ คียส์,491591
5,20180901233542,en,844509740,Robert Forhan,//en.wikipedia.org/wiki/Robert Forhan,"{'any': 0.39078498293515, 'human': 0.322525597...",th,7678635,โรเบิร์ต ฟอร์แฮน,//th.wikipedia.org/wiki/โรเบิร์ต ฟอร์แฮน,491595
6,20180901233521,en,829989743,Amand Dalem,//en.wikipedia.org/wiki/Amand Dalem,"{'any': 0.96084337349398, 'human': 0.960843373...",th,7678839,อามันด์ ดาเลม,//th.wikipedia.org/wiki/อามันด์ ดาเลม,491657
7,20180616124609,en,823655681,Édouard Ferrand,//en.wikipedia.org/wiki/Édouard Ferrand,"{'any': 0.96287703016241, 'human': 0.962877030...",th,7678856,เอดูอาร์ แฟร็อง,//th.wikipedia.org/wiki/เอดูอาร์ แฟร็อง,491660
8,20180901233519,en,823520159,Zeng Jinyan,//en.wikipedia.org/wiki/Zeng Jinyan,"{'any': 0.30705190989226, 'human': 0.215964740...",th,7678890,เจิ้ง จินย่าน,//th.wikipedia.org/wiki/เจิ้ง จินย่าน,491672
9,20180616232534,en,845477568,Jean-Baptiste Kim,//en.wikipedia.org/wiki/Jean-Baptiste Kim,"{'any': 0.617903930131, 'human': 0.61790393013...",th,7679569,จีน-บาติสต์ คิม,//th.wikipedia.org/wiki/จีน-บาติสต์ คิม,491909


#### Explore on stats column

In [6]:
df_stats = df['stats'].apply(pd.Series)
df_stats.head()

,any,human,mt,mtSectionsCount,translatedSectionsCount
0,0.413196,0.313992,0.099204,1.0,NaN
1,0.923833,0.883292,0.040541,1.0,NaN
2,0.428492,0.428492,0.000000,0.0,NaN
3,0.591398,0.479570,0.111828,1.0,NaN
4,0.867140,0.735294,0.131846,1.0,NaN


In [7]:
df_stats.describe()

,any,human,mt,mtSectionsCount,translatedSectionsCount
count,500.000000,500.000000,500.000000,500.000000,4.000000
mean,0.724505,0.628018,0.096488,1.532000,20.750000
std,0.569838,0.583701,0.240336,5.035045,17.783419
min,0.000000,0.000000,0.000000,0.000000,3.000000
25%,0.352576,0.239826,0.000000,0.000000,11.250000
50%,0.574587,0.440298,0.002692,1.000000,17.500000
75%,0.894474,0.834261,0.062125,1.000000,27.000000
max,4.302449,4.302449,2.862802,64.000000,45.000000


In [8]:
df_stats[df_stats['any'] > 2.5]

,any,human,mt,mtSectionsCount,translatedSectionsCount
321,3.530609,3.333557,0.197051,1.0,NaN
378,2.788609,2.788609,0.000000,0.0,NaN
379,2.508139,2.506511,0.001628,1.0,NaN
438,2.906087,0.043285,2.862802,1.0,NaN
440,2.948431,2.948431,0.000000,0.0,NaN
464,2.911345,2.852139,0.059206,1.0,NaN
481,2.570600,2.570600,0.000000,0.0,NaN
485,4.302449,4.302449,0.000000,0.0,NaN


In [9]:
df_stats[df_stats['mt'] > 1]

,any,human,mt,mtSectionsCount,translatedSectionsCount
46,1.090814,0.000000,1.090814,8.0,NaN
419,2.197262,0.060021,2.137241,1.0,NaN
438,2.906087,0.043285,2.862802,1.0,NaN


**Note:** It can be seen that some articles has `any` (total translation percentage) and `mt` (machine translation percentage) higher than 1.
Which [Outreachy mentor explained that the number base on word count.](https://phabricator.wikimedia.org/T218003#5029680) So it is possible that word in source article is higher than translated article, resulted in `any` and `mt` higher than 1.

#### Explore the interesting articles

In [10]:
df[['sourceTitle','targetTitle']][:30]

,sourceTitle,targetTitle
0,Sonia Scurfield,ซอนย่า สเกอร์ฟิลด์
1,Andrew Massey (conductor),แอนดี แมสซีย์ (วาทยากร)
2,Sam Moore (publisher),ซัม มัวร์ (ผู้เผยแพร่)
3,André Desvages,อ็องเดร เดอวาก
4,Johnnie Keyes,จอห์นนี่ คียส์
5,Robert Forhan,โรเบิร์ต ฟอร์แฮน
6,Amand Dalem,อามันด์ ดาเลม
7,Édouard Ferrand,เอดูอาร์ แฟร็อง
8,Zeng Jinyan,เจิ้ง จินย่าน
9,Jean-Baptiste Kim,จีน-บาติสต์ คิม


##### The Holy city article

In [11]:
df[df['sourceTitle'] == 'Holy city']

,publishedDate,sourceLanguage,sourceRevisionId,sourceTitle,sourceURL,stats,targetLanguage,targetRevisionId,targetTitle,targetURL,translationId
20,20180625112801,en,845754238,Holy city,//en.wikipedia.org/wiki/Holy city,"{'any': 0.8185914552737, 'human': 0.8169225634...",th,7694813,เมืองศักดิ์สิทธิ์,//th.wikipedia.org/wiki/เมืองศักดิ์สิทธิ์,492437


In [12]:
holyc_source_title = df.loc[20, 'sourceTitle']
holyc_target_title = df.loc[20, 'targetTitle']
holyc_tid = df.loc[20,'translationId']
holyc_source_revid = df.loc[20, 'sourceRevisionId']
print(holyc_source_title)
print(holyc_target_title)
print(holyc_tid)
print(holyc_source_revid)

Holy city
เมืองศักดิ์สิทธิ์
492437
845754238


## Get corresponding parallel translation

In [13]:
#Download dump to access all translated articles
json_str = ""
with gzip.open('cx-corpora.en2th.text.json.gz', 'rt', encoding='utf-8') as fin:
    for line in fin:
        json_str += line.strip()
# remove repetitive commas
json_str = re.sub(',{2,}', ',', json_str)
parallel_corpus = json.loads(json_str)

In [14]:
print("Descriptive statistics:")
print("{0} translated sections.".format(len(parallel_corpus)))
print("{0} translated articles.".format(len(set(sec['id'].split('/')[0] for sec in parallel_corpus))))

Descriptive statistics:
18300 translated sections.
2670 translated articles.


In [15]:
mt_counts = {}
for sec in parallel_corpus:
    if sec['mt']:
        service = sec['mt']['engine']
        mt_counts[service] = mt_counts.get(service, 0) + 1
    else:
        mt_counts['no-mt'] = mt_counts.get('no-mt', 0) + 1
print("Machine translation services used: {0}".format(mt_counts))
print("\nExample:\n", parallel_corpus[0])

Machine translation services used: {'no-mt': 16474, 'Yandex': 1642, 'Google': 183, 'scratch': 1}

Example:
 {'id': '43176/mwcx-source-title', 'sourceLanguage': 'en', 'targetLanguage': 'th', 'source': {'content': 'M25 motorway'}, 'mt': None, 'target': {'content': 'มอเตอร์เวย์เอ็ม 25'}}


In [16]:
# Translated sections for "Holy city" article
for translated_section in parallel_corpus:
    if translated_section['id'].split('/')[0] == holyc_tid:
        print(translated_section, '\n')

{'id': '492437/69824d3b0e70ca6aaa0da1613b65fd', 'sourceLanguage': 'en', 'targetLanguage': 'th', 'source': {'content': 'References'}, 'mt': {'engine': 'Yandex', 'content': 'อ้างอิง'}, 'target': {'content': 'References'}} 

{'id': '492437/0f5fbbd8e643b1b08a782d4e206f94', 'sourceLanguage': 'en', 'targetLanguage': 'th', 'source': {'content': 'South Asia'}, 'mt': None, 'target': {'content': 'เอเชียใต้'}} 

{'id': '492437/2d1b84c3b2d1353bb8d6c146558555', 'sourceLanguage': 'en', 'targetLanguage': 'th', 'source': {'content': 'Americas'}, 'mt': None, 'target': {'content': 'อเมริกา'}} 

{'id': '492437/37ee626c09bb4ac040663fda7879ea', 'sourceLanguage': 'en', 'targetLanguage': 'th', 'source': {'content': 'West Asia'}, 'mt': None, 'target': {'content': 'เอเชียตะวันตก'}} 

{'id': '492437/6f202f54a7b2d8fe42aa804eabe48e', 'sourceLanguage': 'en', 'targetLanguage': 'th', 'source': {'content': 'List'}, 'mt': None, 'target': {'content': 'รายชื่อเมืองศักดิ์สิทธิ์ทั่วโลก'}} 

{'id': '492437/a5893730881df686

# Analyses

## Qualitative

**TODO**

ดู talk กับ history few articles เปรียบเทียบที่แปลกับต้นฉบับ ณ ขณะนี้

What types of content were added after the translation? 

Are the articles diverging in content or staying similar? 

What sorts of discussions occur on the talk pages of translated articles?

Eventually we can do this in a more robust manner: more carefully choosing which articles to examine, developing more concrete questions to answer, building a code book for annotating article histories, content, or discussions, etc.

 "has new content in the source language been added to the target language"

## Quantitative

In [28]:
# Page revision history for "Holy city" article
revision_parameters = {
    "action": "query",
    "prop": "revisions",
    "titles": holyc_source_title,
    "rvprop": "timestamp|user|size|comment",
    "rvlimit": 100,
    "rvstartid": holyc_source_revid,
    "rvslots": "main",
    "formatversion": "2",
    "format": "json",
    "rvdir": "newer"
}
holyc_revisions = session.get(revision_parameters)
holyc_revisions

{'batchcomplete': True,
 'query': {'pages': [{'pageid': 9288882,
    'ns': 0,
    'title': 'Holy city',
    'revisions': [{'user': 'Occono',
      'timestamp': '2018-06-13T22:08:51Z',
      'size': 8607,
      'comment': 'Reverting Vandalism by [[User:Stanley T. Flomo]]'},
     {'user': '111.92.120.123',
      'anon': True,
      'timestamp': '2018-07-20T15:16:32Z',
      'size': 8597,
      'comment': ''},
     {'user': '175.142.24.96',
      'anon': True,
      'timestamp': '2018-07-23T00:39:27Z',
      'size': 8673,
      'comment': '/* Southeast Asia */'},
     {'user': '175.142.24.96',
      'anon': True,
      'timestamp': '2018-07-23T00:42:10Z',
      'size': 8632,
      'comment': '/* Southeast Asia */'},
     {'user': '175.142.24.96',
      'anon': True,
      'timestamp': '2018-07-23T00:43:15Z',
      'size': 8627,
      'comment': '/* Southeast Asia */'},
     {'user': 'Abraham',
      'timestamp': '2018-08-17T16:44:21Z',
      'size': 8669,
      'comment': '/* Europe */ ci

In [18]:
session_th = mwapi.Session(host='https://th.wikipedia.org',
                        user_agent='mwapi (python) -- outreachy content translation')

In [25]:
# Page revision history for "Holy city"  in target article
target_revision_parameters = {
    "action": "query",
    "prop": "revisions",
    "titles": holyc_target_title,
    "rvprop": "timestamp|user|comment",
    "rvlimit": 100,
    "rvstartid": holyc_tid,
    "rvslots": "main",
    "formatversion": "2",
    "format": "json",
    "rvdir": "newer"
}
holyc_target_revisions = session_th.get(target_revision_parameters)
holyc_target_revisions

{'batchcomplete': True,
 'query': {'pages': [{'pageid': 954923,
    'ns': 0,
    'title': 'เมืองศักดิ์สิทธิ์',
    'revisions': [{'user': 'Potapt',
      'timestamp': '2018-06-29T18:16:22Z',
      'comment': 'Potapt ย้ายหน้า [[เมืองศักดิ์สิทธิ์]] ไปยัง [[นครศักดิ์สิทธิ์]]'}]}]}}

### Page views
compare page views between source and target articles

**TODO**

https://paws-public.wmflabs.org/paws-public/User:Jtmorgan/ds4ux/paws-cheatsheet.ipynb

### Edit
compare number of editing between source and target articles

**TODO**

ดูได้ไหมว่านับจากวันที่ที่แปล มีการ edit กี่ครั้ง คือนับจาก timestamp ตั้งแต่วันที่แปล นับทั้งต้นทางปลายทาง อันไหนที่มี edit รวม (?) มากที่สุด
แล้วค่อยไปดูรายละเอียดว่า article ที่มีการ edit มากๆ ต่างกันมากไหม เป็น edit เล็กๆ หรือใหญ่ๆ

ดูขนาดได้ด้วย

+ loop 0-499

`holyc_source_title = df.loc[0, 'sourceTitle']

holyc_source_revid = df.loc[0, 'sourceRevisionId']

holyc_target_title = df.loc[0, 'targetTitle']

holyc_tid = df.loc[0,'translationId']`

+ result going to be in table, column are
    + sourceTitle
    + sourceRevisionId
    + targetTitle
    + translationId
    + date of creating target article
    + source edit amount since it got translated
    + target edit amount
    + source size change since it got translated
    + target size change
+ count how many times it got edit on both articles
+ เช็คว่ามีการย้ายหน้าได้ไหม

In [20]:
#count editing time
len(holyc_revisions['query']['pages'][0]['revisions']) 

20

In [42]:
#the first timestamp
holyc_revisions['query']['pages'][0]['revisions'][0]['timestamp']

'2018-06-13T22:08:51Z'

In [49]:
'ย้ายหน้า' in holyc_target_revisions['query']['pages'][0]['revisions'][0]['comment']

True

In [45]:
#create emty dict for add data
edit_n_size_df = {'sourceTitle':[],
                            'sourceRevisionId':[],
                            'targetTitle':[],
                            'translationId':[],
                            'dateCreatingTarget':[],
                            'sourceEdit':[],
                            'targetEdit':[],
                            'sourceSizeChange':[],
                            'targetSizeChange':[]
                            }

In [43]:
#loop through 500 articles
for i in range(0,5):
    source_title = df.loc[i, 'sourceTitle']
    source_revid = df.loc[i, 'sourceRevisionId']
    target_title = df.loc[i, 'targetTitle']
    tid = df.loc[i,'translationId']
    
    # create paremeters for collect revision data from source articles
    revision_parameters_s = {
        "action": "query",
        "prop": "revisions",
        "titles": source_title,
        "rvprop": "timestamp|size",
        "rvlimit": 100,
        "rvstartid": source_revid,
        "rvslots": "main",
        "formatversion": "2",
        "format": "json",
        "rvdir": "newer"
        }
    # revision data from source articles
    source_revisions = session.get(revision_parameters_s)
    
    # create paremeters for collect revision data from target articles
    revision_parameters_t = {
        "action": "query",
        "prop": "revisions",
        "titles": target_title,
        "rvprop": "timestamp|size",
        "rvlimit": 100,
        "rvstartid": tid,
        "rvslots": "main",
        "formatversion": "2",
        "format": "json",
        "rvdir": "newer"
        }
    # revision data from target articles
    target_revisions = session_th.get(revision_parameters_t)
    
    # collect date of creating target article
    date_c_target = target_revisions['query']['pages'][0]['revisions'][0]['timestamp']
    
    # TODO count source edit amount since it got translated
    # TODO count target edit amount
    # TODO count source edit amount since it got translated
    # TODO count source edit amount since it got translated
    
    # append data to dictionary
    edit_n_size_df['sourceTitle'].append(source_title)
    edit_n_size_df['sourceRevisionId'].append(source_revid)
    edit_n_size_df['targetTitle'].append(target_title)
    edit_n_size_df['translationId'].append(tid)
    edit_n_size_df['translationId'].append(date_c_target)

In [44]:
edit_n_size_df

{'sourceTitle': ['Sonia Scurfield',
  'Andrew Massey (conductor)',
  'Sam Moore (publisher)',
  'André Desvages',
  'Johnnie Keyes'],
 'sourceRevisionId': ['846093806',
  '844984088',
  '845176394',
  '845016481',
  '845590955'],
 'targetTitle': ['ซอนย่า สเกอร์ฟิลด์',
  'แอนดี แมสซีย์ (วาทยากร)',
  'ซัม มัวร์ (ผู้เผยแพร่)',
  'อ็องเดร เดอวาก',
  'จอห์นนี่ คียส์'],
 'translationId': ['491563',
  '2018-06-16T09:03:18Z',
  '491571',
  '2018-06-16T09:25:27Z',
  '491580',
  '2018-06-16T09:34:31Z',
  '491589',
  '2018-06-16T09:47:32Z',
  '491591',
  '2018-06-16T09:59:38Z'],
 'dateCreatingTarget': [],
 'sourceEdit': [],
 'targetEdit': [],
 'sourceSizeChange': [],
 'targetSizeChange': []}

In [31]:
edit_n_size_pd = pd.DataFrame(edit_n_size_df)

ValueError: arrays must all be same length